In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
from mne.preprocessing import corrmap
import pickle as pkl

from Cho2017 import SubjectData

pickle_cache_dir = 'denoise_classify_data'
subject_data_pickle_path = lambda n: os.path.join(pickle_cache_dir, 'data_{}.pkl'.format(n))
subject_ica_pickle_path = lambda n: os.path.join(pickle_cache_dir, 'ica_{}.pkl'.format(n))


plt.rcParams['figure.figsize'] = [12, 8]

# # Make sure ipympl is installed (via pip) for interactive plots in Jupyter 
# %matplotlib widget

In [2]:
ICA_SEED = 97 # Just some random seed
CORRMAP_THRESH = 0.9
n_ica_comps = 64
n_template_ica_comps = 10

In [3]:
exclude_ica_components = {}

for subj_n in range(1, 53):
    print("\n\n========================================")
    print('Subject {:02d}'.format(subj_n))
    print("========================================")
    if os.path.exists(subject_data_pickle_path(subj_n)):
        with open(subject_data_pickle_path(subj_n), 'rb') as f:
            print('Loading', subject_data_pickle_path(subj_n))
            subject = pkl.load(f)
    else:
        subject = SubjectData('Cho2017-Data/s{:02d}.mat'.format(subj_n))
        with open(subject_data_pickle_path(subj_n), 'wb') as f:
            print('Reading', subject_datapickle_path(subj_n))
            pkl.dump(subject, f)
    
    im_left_epochs = subject.get_epochs('imagery_left', tmin=-0.5, tmax=2, picks=['eeg'], filter_freqs=(1, None), reject_criteria=None, verbose=False)
    im_right_epochs = subject.get_epochs('imagery_right', tmin=-0.5, tmax=2, picks=['eeg'], filter_freqs=(1, None), reject_criteria=None, verbose=False)
    
    
    
    ############################################################
    # 
    # Get motor imagery ICA components 
    #
    ############################################################
    joint_im_epochs = mne.concatenate_epochs([im_left_epochs.copy(), im_right_epochs.copy()], add_offset=True)
    labels = np.hstack((im_left_epochs.events[:, -1] - 1, im_right_epochs.events[:, -1]))  # Set left events to 0 to have 2 distinct class labels (left: 0 and right: 1)
    
    if os.path.exists(subject_ica_pickle_path(subj_n)):
        with open(subject_ica_pickle_path(subj_n), 'rb') as f:
            print('Loading', subject_ica_pickle_path(subj_n))
            ica_joint = pkl.load(f)
    else:
        ica_joint = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=ICA_SEED, max_iter=1000)
        ica_joint.fit(joint_im_epochs)
        with open(subject_ica_pickle_path(subj_n), 'wb') as f:
            print('Writing', subject_ica_pickle_path(subj_n))
            pkl.dump(ica_joint, f)

    
    # ica_joint.plot_components();
    
    
    ############################################################
    # 
    # Get blinking artifact ICA components 
    #
    ############################################################
    picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

    # Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
    blinking = [subject.raw_noise['blinking'].copy().crop(tmin=0, tmax=5), subject.raw_noise['blinking'].copy().crop(tmin=5, tmax=None)]
    # blinking[0].copy().pick(picks).plot();
    # blinking[1].copy().pick(picks).plot();

    find_eog_ch = 'FP1'
    tmin = -0.25
    tmax = 0.25
    baseline_toffset = 0.01

    trial0_blink_events = find_eog_events(blinking[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
    trial1_blink_events = find_eog_events(blinking[1], ch_name=find_eog_ch, filter_length=2560)
    
    filter_freqs = (1, None)
    filter_props = dict(picks=['eeg'], fir_design='firwin', skip_by_annotation='edge')
    trial0_blink_epochs = mne.Epochs(blinking[0], trial0_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)
    trial1_blink_epochs = mne.Epochs(blinking[1], trial1_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)
    # trial0_blink_epochs.plot(picks=picks);
    # trial1_blink_epochs.plot(picks=picks);

    blink_epochs = concatenate_epochs([trial0_blink_epochs, trial1_blink_epochs])
    # blink_epochs.plot(picks=picks);

    ica_blink = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=ICA_SEED, max_iter=1000)
    ica_blink.fit(blink_epochs)
    # ica_blink.plot_components();
    
   
    ############################################################
    # 
    # Get Correlation map across MI/blinking ICA components
    #
    ############################################################
    for i in range(n_template_ica_comps):   # ICA components are sorted by explained variange (R^2), so we only need to take the first few components
        try:
            corrmap([ica_joint], ica_blink.get_components()[:, i], label='test', show=False, plot=False, threshold=CORRMAP_THRESH)
        except Exception as e:
            print(e)
    
    print(ica_joint.labels_['test'])
    ica_joint.plot_components(picks=ica_joint.labels_['test']);
    exclude_ica_components[subj_n] = ica_joint.labels_['test']



Subject 49
Loading denoise_classify_data/data_49.pkl
100 events found
Event IDs: [1]


/mnt/Projects/mlsp/18-797_Semester_Project/Cho2017.py:157: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  epoch = epoch.filter(*filter_freqs, **filter_props)


100 events found
Event IDs: [1]


/mnt/Projects/mlsp/18-797_Semester_Project/Cho2017.py:157: RuntimeWarning: filter_length (1691) is longer than the signal (1281), distortion is likely. Reduce filter length or filter a longer signal.
  epoch = epoch.filter(*filter_freqs, **filter_props)


Loading denoise_classify_data/ica_49.pkl
Using channel FP1 as EOG channel
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2561 samples (5.002 sec)

Now detecting blinks and generating corresponding events
Found 6 significant peaks
Number of EOG events detected : 6
Using channel FP1 as EOG channel
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz



<ipython-input-3-f7c8e4b48677>:67: RuntimeWarning: filter_length (1691) is longer than the signal (257), distortion is likely. Reduce filter length or filter a longer signal.
  trial0_blink_epochs = mne.Epochs(blinking[0], trial0_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)
<ipython-input-3-f7c8e4b48677>:68: RuntimeWarning: filter_length (1691) is longer than the signal (257), distortion is likely. Reduce filter length or filter a longer signal.
  trial1_blink_epochs = mne.Epochs(blinking[1], trial1_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)


Fitting ICA took 2.3s.
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a more lenient threshold
No component detected using when z-scoring threshold [0.9], consider using a 

KeyError: 'test'

In [ ]:
print(exclude_ica_components)

In [ ]:
import pickle as pkl
    
with open('ica_excludes_SEED-{}_THRESH-{}.pkl'.format(ICA_SEED, CORRMAP_THRESH), 'wb') as f:
    pkl.dump(exclude_ica_components, f)

In [6]:
import pickle as pkl


with open('ica_excludes_SEED-{}_THRESH-{}.pkl'.format(ICA_SEED, CORRMAP_THRESH), 'rb') as f:
    _exclude_ica_components = pkl.load(f)

    

FileNotFoundError: [Errno 2] No such file or directory: 'ica_excludes_SEED-97_THRESH-0.8.pkl'